In [ ]:
from pipeline.text2sql_enricher import Text2SQLPipeline

/Users/sinabehnam/Desktop/Projects/Polito/Thesis/MA_text2SQL/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
import json
import glob
import traceback
from datetime import datetime

BIRD_DATASET_PATH = '/MA_text2SQL/DataSampling/src/src_data/bird_subset/stratified_output'
SPIDER_DATASET_PATH = '/MA_text2SQL/DataSampling/src/src_data/spider_subset/spider_stratified_output_200'
API_KEY_PATH = '/MA_text2SQL/Data/anthropic.api.key/text2sql.key'

# Load the API key
API_KEY = Text2SQLPipeline._get_together_api_key(API_KEY_PATH)

In [3]:
# Set up output directories for updated JSON files
OUTPUT_BASE_DIR = 'entriched_full_dataset_best_1'
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

# Define model configurations for testing
model_configs = [
    # API-based models
    # {
    #     "type": "together_ai",
    #     "name": "deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    #     "api_key": API_KEY
    # },
    # {
    #     "type": "together_ai",
    #     "name": "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    #     "api_key": API_KEY,
    # },
    # Anthropic models 
    {
        "type": "anthropic",
        "name": "claude-3-7-sonnet-20250219",
        "api_key": API_KEY,
        "extended_thinking": True,
    },
    # Local models - uncomment and configure as needed
    # {
    #     "type": "local",
    #     "name": "IceKingBing/SQL-Llama-v0.5",
    #     "device": "auto",
    #     "max_new_tokens": 1024
    # },
    # {
    #     "type": "local",
    #     "name": "TheBloke/Llama-2-7B-Chat-GGUF",
    #     "device": "cpu",
    #     "max_new_tokens": 512
    # }
]

In [4]:
all_results = {}

for model_config in model_configs:
    model_name = model_config["name"]
    model_type = model_config["type"]

    try:
        print(f"\nEvaluating {model_type} model: {model_name}")

        # Create model-specific output directory
        # Use the last part of the model name for the directory name
        model_dir_name = model_name.split('/')[-1]
        model_output_dir = os.path.join(OUTPUT_BASE_DIR, model_dir_name)
        os.makedirs(model_output_dir, exist_ok=True)

        # Initialize the pipeline with this model configuration
        pipeline = Text2SQLPipeline(
            bird_path=BIRD_DATASET_PATH,
            spider_path=SPIDER_DATASET_PATH,
            model_config=model_config
        )

        # Run the pipeline
        results = pipeline.run_pipeline(
            save_updated_files=True,
            output_dir=model_output_dir
        )

        # Store summary metrics (without detailed results)
        summary_metrics = {
            'num_evaluated': results['num_evaluated'],
            'num_with_prediction': results['num_with_prediction'],
            'prediction_rate': results['prediction_rate'],
            'execution_accuracy': results['execution_accuracy'],
            'exact_match_accuracy': results['exact_match_accuracy'],
            'semantic_equivalent_accuracy': results.get('semantic_equivalent_accuracy', 0.0),
            'model': results['model']
        }

        all_results[model_name] = summary_metrics

        # Print summary metrics
        print("\nSummary Metrics:")
        print(f"Total evaluated: {summary_metrics['num_evaluated']}")
        print(f"Prediction rate: {summary_metrics['prediction_rate']:.2f}")
        print(f"Execution accuracy: {summary_metrics['execution_accuracy']:.2f}")
        print(f"Exact match accuracy: {summary_metrics['exact_match_accuracy']:.2f}")
        print(f"Semantic equivalence accuracy: {summary_metrics.get('semantic_equivalent_accuracy', 0.0):.2f}")
        print(f"Updated JSON files saved to: {model_output_dir}")

    except Exception as e:
        error_traceback = traceback.format_exc()
        print(f"Error processing model {model_name}: {str(e)}")
        print(f"Traceback: {error_traceback}")

        # Add error information to results
        all_results[model_name] = {
            'error': str(e),
            'traceback': error_traceback,
            'model': {'model_name': model_name, 'model_type': model_type}
        }

INFO:text2sql_pipeline:Initializing Text2SQLPipeline...
INFO:text2sql_pipeline:Total data points: 521
INFO:text2sql_pipeline:Bird data points: 200
INFO:text2sql_pipeline:Spider data points: 321
INFO:text2sql_pipeline:Training data points: 521
INFO:text2sql_pipeline:Testing data points: 0



Evaluating anthropic model: claude-3-7-sonnet-20250219


Processing instances:   0%|          | 0/521 [00:00<?, ?instance/s]INFO:text2sql_pipeline:Processing instance 833...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:text2sql_pipeline:Execution correct: True
INFO:text2sql_pipeline:Exact match: True
INFO:text2sql_pipeline:Semantic equivalent: True
INFO:text2sql_pipeline:--------------------------------------------------
Processing instances:   0%|          | 1/521 [00:10<1:31:01, 10.50s/instance]INFO:text2sql_pipeline:Processing instance 188...
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:text2sql_pipeline:Execution correct: False
INFO:text2sql_pipeline:Exact match: False
INFO:text2sql_pipeline:Semantic equivalent: False
INFO:text2sql_pipeline:--------------------------------------------------
Processing instances:   0%|          | 2/521 [00:23<1:45:37, 12.21s/instance]INFO:text2sql_pipeline:Processing instance 537...
INFO:httpx:HTTP Request: POST ht


Summary Metrics:
Total evaluated: 521
Prediction rate: 1.00
Execution accuracy: 0.64
Exact match accuracy: 0.05
Semantic equivalence accuracy: 0.68
Updated JSON files saved to: entriched_full_dataset_best_1/claude-3-7-sonnet-20250219


In [5]:
# Save only summary metrics to file
summary_file = os.path.join(OUTPUT_BASE_DIR, 'entriched_full_dataset_best_1_summary.json')
with open(summary_file, 'w') as f:
    json.dump(all_results, f, indent=2)

print(f"\nEvaluation complete. Summary saved to '{summary_file}'")
print(f"Individual JSON files with inference results saved to model-specific directories under '{OUTPUT_BASE_DIR}'")


Evaluation complete. Summary saved to 'entriched_full_dataset_best_1/entriched_full_dataset_best_1_summary.json'
Individual JSON files with inference results saved to model-specific directories under 'entriched_full_dataset_best_1'


In [ ]:
def generate_evaluation_summary_from_directory(base_dir):
    """
    Generate evaluation summary by reading enriched JSON files from directory.
    
    Args:
        base_dir: Base directory containing model-specific subdirectories with enriched JSON files
        
    Returns:
        Dictionary with evaluation metrics for each model
    """
    summary = {}
    
    # Find all model directories
    model_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    
    for model_dir in model_dirs:
        model_path = os.path.join(base_dir, model_dir)
        
        # Get all JSON files in this model directory
        json_files = glob.glob(os.path.join(model_path, '*.json'))
        
        if not json_files:
            print(f"No JSON files found in {model_path}")
            continue
        
        # Metrics to calculate
        num_evaluated = len(json_files)
        has_prediction = 0
        execution_correct = 0
        exact_match = 0
        semantic_equivalent = 0
        model_info = None
        
        # Process each JSON file
        for json_file in json_files:
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                
                # Extract inference results
                if 'inference_results' in data:
                    inference = data['inference_results']
                    
                    # Save model info if not already set
                    if model_info is None and 'model' in inference:
                        model_info = inference['model']
                    
                    # Count metrics
                    if inference.get('has_prediction', False):
                        has_prediction += 1
                        
                        pred_output = inference.get('predicted_output', {})
                        if pred_output.get('execution_correct', False):
                            execution_correct += 1
                        if pred_output.get('exact_match', False):
                            exact_match += 1
                        if pred_output.get('semantic_equivalent', False):
                            semantic_equivalent += 1
            
            except Exception as e:
                print(f"Error processing {json_file}: {str(e)}")
        
        # Calculate metrics
        metrics = {
            'num_evaluated': num_evaluated,
            'num_with_prediction': has_prediction,
            'prediction_rate': has_prediction / num_evaluated if num_evaluated > 0 else 0,
            'execution_accuracy': execution_correct / has_prediction if has_prediction > 0 else 0,
            'exact_match_accuracy': exact_match / has_prediction if has_prediction > 0 else 0,
            'semantic_equivalent_accuracy': semantic_equivalent / has_prediction if has_prediction > 0 else 0,
            'model': model_info or {'model_name': model_dir}
        }
        
        # Get the full model name from the model info
        model_name = model_info['model_name'] if model_info and 'model_name' in model_info else model_dir
        
        # Add to summary
        summary[model_name] = metrics
        
        # Print metrics
        print(f"\nMetrics for model {model_name}:")
        print(f"Total evaluated: {metrics['num_evaluated']}")
        print(f"Prediction rate: {metrics['prediction_rate']:.2f}")
        print(f"Execution accuracy: {metrics['execution_accuracy']:.2f}")
        print(f"Exact match accuracy: {metrics['exact_match_accuracy']:.2f}")
        print(f"Semantic equivalence accuracy: {metrics['semantic_equivalent_accuracy']:.2f}")
    
    return summary

In [ ]:
import json
import os
import glob
# Alternatively, generate summary from enriched files
print("\nGenerating summary from enriched files...")
dir_path = '/MA_text2SQL/DataSampling/src/models/pipeline/entriched_full_dataset_1'
directory_summary = generate_evaluation_summary_from_directory(dir_path)

# Save the directory-based summary
summary_from_dir_file = os.path.join(dir_path, 'evaluation_summary_from_dir.json')
with open(summary_from_dir_file, 'w') as f:
    json.dump(directory_summary, f, indent=2)

print(f"Summary from directory saved to '{summary_from_dir_file}'")